In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy.stats import percentileofscore as score




pd.options.mode.copy_on_write = True


In [31]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

sp500_df = pd.read_html(url)[0]
sp500_df

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [26]:
stocks_to_remove = ['BF.B', 'BRK.B']

In [32]:
sp500_df = sp500_df[~(sp500_df['Symbol'].isin(stocks_to_remove))]
symbol_security = sp500_df[['Symbol', 'Security']]


tickers = sp500_df['Symbol'].tolist()
company_names = sp500_df['Security'].tolist()

In [34]:
sp500_df[sp500_df['Symbol'].isin(stocks_to_remove)]

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded


The dataframe must have the columns Ticker, Price, 1-Year Price return, 6-months Price return, 3-months Price return, 1-month Price return, Number of Shares to Buy, return percentiles, and finally the mean score of the four time periods.

In [46]:

def price_return(data, curr_price, period):

                  #   6 months
  past_date = data['Date'].iloc[-1] - pd.DateOffset(months = period)


            # Check if the past date is in the DataFrame

  past_price = data.loc[data['Date'] == past_date, 'Close']

  if not past_price.empty:
      # Past date found, use the price
      past_price_value = past_price.values[0]
  else:
      # Past date not found, find the most recent date before the past date
      recent_dates = data[data['Date'] < past_date]
      if not recent_dates.empty:
          past_price_value = recent_dates['Close'].iloc[-1]
      else:
          past_price_value = np.nan

  # Calculate the return if past_price_value is not NaN
  if not np.isnan(past_price_value):
      return_value = (curr_price - past_price_value) / past_price_value * 100
  else:
      return_value = np.nan

  return return_value



#       FUNCTION TO GET STOCK DATA
def get_stock_data(tickers):
  data = pd.DataFrame(columns = ['Ticker', 'Price', '1-year Price Return', '6-months Price Return', '3-months Price Return', '1-month Price Return', '1-year Price Return Percentile', '6-months Price Return Percentile'	, '3-months Price Return Percentile', '1-month Price Return Percentile'])

  for ticker in tickers:
    ticker = yf.Ticker(ticker)

    df = ticker.history(period = 'max')
    df.reset_index(inplace = True)



    #   CALCULATE PRICE RETURNS



                    #   1 year
    current_price = df['Close'].iloc[-1]
    one_year_past_price_value = df['Close'].iloc[0]
    one_year_price_return_value = (current_price - one_year_past_price_value) / one_year_past_price_value * 100






    # append values



    return_value_6_mnths = price_return(df, current_price, 6)
    return_value_3_mnths = price_return(df, current_price, 3)
    return_value_1_mnth = price_return(df, current_price, 1)

    time_periods = ["1-year Price Return",	"6-months Price Return", "3-months Price Return", "1-month Price Return"]




    data.loc[len(data)] = [ticker.ticker, current_price, one_year_price_return_value, return_value_6_mnths, return_value_3_mnths, return_value_1_mnth, 'NA', 'NA', 'NA', 'NA']

    for row in data.index:
      for time_period in time_periods:
        percentile_col = f"{time_period} Percentile"
        data.loc[row, percentile_col] = score(data[time_period], data.loc[row, time_period])

  return data

In [48]:
stock_data = get_stock_data(tickers)

In [52]:
stock_data.isnull().sum()


,0
Ticker,0
Price,0
1-year Price Return,0
6-months Price Return,2
3-months Price Return,1
1-month Price Return,0
1-year Price Return Percentile,0
6-months Price Return Percentile,501
3-months Price Return Percentile,501
1-month Price Return Percentile,0


In [53]:
stock_data[stock_data['6-months Price Return'].isnull() == True]

,Ticker,Price,1-year Price Return,6-months Price Return,3-months Price Return,1-month Price Return,1-year Price Return Percentile,6-months Price Return Percentile,3-months Price Return Percentile,1-month Price Return Percentile
24,AMTM,23.780001,-19.417145,NaN,NaN,-5.634921,0.798403,NaN,NaN,17.165669
416,SW,54.700001,19.945450,NaN,17.764118,5.884632,2.195609,NaN,NaN,85.02994


# TEST

In [ ]:
stco = 'BRK.B'

ticker = yf.Ticker(stco)

df = ticker.history(period = 'max')
df.reset_index(inplace = True)


ERROR:yfinance:$BRK.B: possibly delisted; no timezone found


Berkshire Hathaway Inc. (BRK.B) has been delisted. SO I have to drop it.

In [37]:
dataf = pd.DataFrame(columns = ['Name', 'Age'])
dataf

,Name,Age


In [39]:
dataf.loc[len(dataf)]

KeyError: 0